
**Load Libraries**

In [2]:
#Load libraries
import tweepy
import requests
from tweepy import OAuthHandler
import os
import json
from timeit import default_timer as timer
import random
import sys
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

## STEP 1: DATA GATHERING

In [3]:
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [4]:
twitter_archive.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [5]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
my_download = requests.get(url)
my_download

<Response [200]>

In [6]:
with open('image-predictions.tsv', 'wb') as file:
    file.write(my_download.content)

In [7]:
image_prediction = pd.read_csv('image-predictions.tsv', sep='\t')
image_prediction.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [8]:
CONSUMER_KEY = 'YOUR CONSUMER KEY'
CONSUMER_SECRET = 'YOUR CONSUMER SECRET'
OAUTH_TOKEN = 'YOUR ACCESS TOKEN'
OAUTH_TOKEN_SECRET = 'YOUR ACCESS SECRET'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

api = tweepy.API(auth, wait_on_rate_limit = True)

In [9]:
#NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
# df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
# change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv
# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API

In [10]:
tweet_ids = twitter_archive.tweet_id.values
len(tweet_ids)


2356

In [11]:
# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()

In [11]:
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.errors.TweepyException as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

### The dataset has been downloaded already and written into the tweet_json.txt file. The data will be extracted from this file below

In [12]:
tweets = []
for line in open('tweet_json.txt', 'r'):
    tweets.append(json.loads(line))
#print out one json object for further reference 
tweets[0] 

{'created_at': 'Tue Aug 01 16:23:56 +0000 2017',
 'id': 892420643555336193,
 'id_str': '892420643555336193',
 'full_text': "This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU",
 'truncated': False,
 'display_text_range': [0, 85],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 892420639486877696,
    'id_str': '892420639486877696',
    'indices': [86, 109],
    'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'url': 'https://t.co/MgUWQ76dJU',
    'display_url': 'pic.twitter.com/MgUWQ76dJU',
    'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1',
    'type': 'photo',
    'sizes': {'large': {'w': 540, 'h': 528, 'resize': 'fit'},
     'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'small': {'w': 540, 'h': 528, 'resize': 'fit'},
     'medium': {'w': 

In [13]:
# Defining the properties that are needed.
vital_properties = ['id', 'favorite_count', 'retweet_count']

In [14]:
#Create an empty list to
# Make use of the Dictionary
archived_tweets = []
with open('tweet_json.txt', 'r') as r: 
    json_tweet = r.readline()
    while json_tweet:
        dictionnary = json.loads(json_tweet)
        data_row = dict((i, dictionnary[i]) for i in vital_properties)
        archived_tweets.append(data_row)
        json_tweet = r.readline()

In [15]:
# create DataFrame    
df_tweets = pd.DataFrame.from_dict(archived_tweets)

In [16]:
df_tweets.head()

,id,favorite_count,retweet_count
0,892420643555336193,39467,8853
1,892177421306343426,33819,6514
2,891815181378084864,25461,4328
3,891689557279858688,42908,8964
4,891327558926688256,41048,9774


### 2 Assessing the Data

**2.1 Assessing the archive**

In [17]:
twitter_archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None


### Observation
<li>The Columns doggo, floofer, pupper and puppo all represent one single variable.</li>
<li>The source contains html residues.</li>

In [18]:
twitter_archive.shape

(2356, 17)

In [19]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

### Observation
timestamp column is a string rather than a datetime

In [20]:

# take a data frame column and a column name and print the number of 'None' string in the column
def none_find(column,name):
    
    count=0
    for word in column:
        if word == 'None':
            count+=1
    print('Name of the column is: ', name, ' and has ',count,' number of None.')

In [21]:
none_find(twitter_archive['tweet_id'],'tweet_id')
none_find(twitter_archive['timestamp'],'timestamp')
none_find(twitter_archive['source'],'source')
none_find(twitter_archive['expanded_urls'],'expanded_urls')
none_find(twitter_archive['rating_numerator'],'rating_numerator')
none_find(twitter_archive['rating_denominator'],'rating_denominator')
none_find(twitter_archive['name'],'name')
none_find(twitter_archive['doggo'],'doggo')
none_find(twitter_archive['floofer'],'floofer')
none_find(twitter_archive['pupper'],'pupper')
none_find(twitter_archive['puppo'],'puppo')

Name of the column is:  tweet_id  and has  0  number of None.
Name of the column is:  timestamp  and has  0  number of None.
Name of the column is:  source  and has  0  number of None.
Name of the column is:  expanded_urls  and has  0  number of None.
Name of the column is:  rating_numerator  and has  0  number of None.
Name of the column is:  rating_denominator  and has  0  number of None.
Name of the column is:  name  and has  745  number of None.
Name of the column is:  doggo  and has  2259  number of None.
Name of the column is:  floofer  and has  2346  number of None.
Name of the column is:  pupper  and has  2099  number of None.
Name of the column is:  puppo  and has  2326  number of None.


### Observation
<li>The "name" column has a total of 745  'None' strings present</li>
<li>The "doggo" column has a total of  2259 'None' strings present</li>
<li>The "floofer" column has a total of 2346 'None' strings present</li>
<li>The "pupper" column has a total of 2099 'None' strings present</li>
<li>The "puppo" column has a total of2326 'None' strings present</li>

In [22]:
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS

In [23]:
# take a data frame column and a column name and print the number of stop words in the column
def the_stopwords(column,name):
    count=0
    for word in column:
        if word in ENGLISH_STOP_WORDS:
            count+=1
    print('Name of the column is: ',name,'  and has ',count,' stop words.')

In [24]:
the_stopwords(twitter_archive['tweet_id'],'tweet_id')
the_stopwords(twitter_archive['timestamp'],'timestamp')
the_stopwords(twitter_archive['source'],'source')
the_stopwords(twitter_archive['expanded_urls'],'expanded_urls')
the_stopwords(twitter_archive['rating_numerator'],'rating_numerator')
the_stopwords(twitter_archive['rating_denominator'],'rating_denominator')
the_stopwords(twitter_archive['name'],'name')
the_stopwords(twitter_archive['doggo'],'doggo')
the_stopwords(twitter_archive['floofer'],'floofer')
the_stopwords(twitter_archive['pupper'],'pupper')
the_stopwords(twitter_archive['puppo'],'puppo')

Name of the column is:  tweet_id   and has  0  stop words.
Name of the column is:  timestamp   and has  0  stop words.
Name of the column is:  source   and has  0  stop words.
Name of the column is:  expanded_urls   and has  0  stop words.
Name of the column is:  rating_numerator   and has  0  stop words.
Name of the column is:  rating_denominator   and has  0  stop words.
Name of the column is:  name   and has  87  stop words.
Name of the column is:  doggo   and has  0  stop words.
Name of the column is:  floofer   and has  0  stop words.
Name of the column is:  pupper   and has  0  stop words.
Name of the column is:  puppo   and has  0  stop words.


### Observation
<li>The "name" column has a total of 87 stop words present</li>

In [48]:
twitter_archive.expanded_urls.sample(5)

829     https://twitter.com/dog_rates/status/700143752...
750     https://twitter.com/dog_rates/status/779123168...
1914                                                  NaN
1261    https://twitter.com/dog_rates/status/710153181...
1371    https://twitter.com/dog_rates/status/702276748...
Name: expanded_urls, dtype: object

In [49]:
twitter_archive.expanded_urls[1261]

'https://twitter.com/dog_rates/status/710153181850935296/photo/1,https://twitter.com/dog_rates/status/710153181850935296/photo/1'

### Observation 
 we notice a Double-Link in "expanded_urls" column data content

In [50]:
#Count the number of entries with double urls
twitter_archive.expanded_urls.str.contains(r',').sum()

639

### Observation 
 The total counts of double url is 639

In [69]:
twitter_archive.rating_denominator.sample(5)  

902     150
1821     10
1924     10
1712     10
1116     10
Name: rating_denominator, dtype: int64

In [70]:
twitter_archive.rating_denominator[902]

150

In [71]:
#find the no. of denominator not equal to 10
num_of_denom = 0
for i in twitter_archive['rating_denominator']:
    if i != 10 :
        num_of_denom += 1
        
print('The count of Denominator not equal to 10 is {}.'.format(num_of_denom))

The count of Denominator not equal to 10 is 23.


### Observation 
Some denomenators are not equal to 10 and they are 23 in number

In [72]:
twitter_archive.rating_numerator.sample(5)

1842    11
532     12
1328    12
1766    12
1781     9
Name: rating_numerator, dtype: int64

In [73]:
# Find the number of values for which the Numerator is less than 10 

num_of_numerator = 0
for num in twitter_archive['rating_numerator']:
    if num <= 9:
        num_of_numerator += 1
        
# Output the Count         
print('The count of Numerator values less than 10 is {}.'.format(num_of_numerator))

The count of Numerator values less than 10 is 440.


### Observation
There exist some numerators less than 10 and they are 440 in number.

## 2.2 Assessing image

In [74]:
image_prediction

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


### Observation
We can observe that the columns "p1", "p2" and "p3" contradict each other.

In [75]:
image_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


### Observations
Column Names are not decriptive enough

In [76]:
image_prediction.duplicated().sum()

0

In [77]:
# Getting the count of image

image_prediction['img_num'].value_counts()

1    1780
2     198
3      66
4      31
Name: img_num, dtype: int64

In [78]:
# take a data frame column and a column name and print the number of 'None' string in the column
none_find(image_prediction['tweet_id'],'tweet_id')
none_find(image_prediction['jpg_url'],'jpg_url')
none_find(image_prediction['img_num'],'img_num')
none_find(image_prediction['p1'],'p1')
none_find(image_prediction['p1_conf'],'p1_conf')
none_find(image_prediction['p2'],'p2')
none_find(image_prediction['p2_conf'],'p2_conf')
none_find(image_prediction['p3'],'p3')
none_find(image_prediction['p3_conf'],'p3_conf')
none_find(image_prediction['p1_dog'],'p1_dog')
none_find(image_prediction['p2_dog'],'p2_dog')
none_find(image_prediction['p3_dog'],'p3_dog')

Name of the column is:  tweet_id  and has  0  number of None.
Name of the column is:  jpg_url  and has  0  number of None.
Name of the column is:  img_num  and has  0  number of None.
Name of the column is:  p1  and has  0  number of None.
Name of the column is:  p1_conf  and has  0  number of None.
Name of the column is:  p2  and has  0  number of None.
Name of the column is:  p2_conf  and has  0  number of None.
Name of the column is:  p3  and has  0  number of None.
Name of the column is:  p3_conf  and has  0  number of None.
Name of the column is:  p1_dog  and has  0  number of None.
Name of the column is:  p2_dog  and has  0  number of None.
Name of the column is:  p3_dog  and has  0  number of None.


In [79]:
#take a data frame column and a column name and print the number of stop words in the column
the_stopwords(image_prediction['tweet_id'],'tweet_id')
the_stopwords(image_prediction['jpg_url'],'jpg_url')
the_stopwords(image_prediction['img_num'],'img_num')
the_stopwords(image_prediction['p1'],'p1')
the_stopwords(image_prediction['p1_conf'],'p1_conf')
the_stopwords(image_prediction['p2'],'p2')
the_stopwords(image_prediction['p2_conf'],'p2_conf')
the_stopwords(image_prediction['p3'],'p3')
the_stopwords(image_prediction['p3_conf'],'p3_conf')
the_stopwords(image_prediction['p1_dog'],'p1_dog')
the_stopwords(image_prediction['p2_dog'],'p2_dog')
the_stopwords(image_prediction['p3_dog'],'p3_dog')

Name of the column is:  tweet_id   and has  0  stop words.
Name of the column is:  jpg_url   and has  0  stop words.
Name of the column is:  img_num   and has  0  stop words.
Name of the column is:  p1   and has  0  stop words.
Name of the column is:  p1_conf   and has  0  stop words.
Name of the column is:  p2   and has  0  stop words.
Name of the column is:  p2_conf   and has  0  stop words.
Name of the column is:  p3   and has  0  stop words.
Name of the column is:  p3_conf   and has  0  stop words.
Name of the column is:  p1_dog   and has  0  stop words.
Name of the column is:  p2_dog   and has  0  stop words.
Name of the column is:  p3_dog   and has  0  stop words.


## 2.3 Assessing data from Twitter API

In [80]:
df_tweets

,id,favorite_count,retweet_count
0,892420643555336193,39467,8853
1,892177421306343426,33819,6514
2,891815181378084864,25461,4328
3,891689557279858688,42908,8964
4,891327558926688256,41048,9774
...,...,...,...
2349,666049248165822465,111,41
2350,666044226329800704,311,147
2351,666033412701032449,128,47
2352,666029285002620928,132,48


In [81]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   id              2354 non-null   int64
 1   favorite_count  2354 non-null   int64
 2   retweet_count   2354 non-null   int64
dtypes: int64(3)
memory usage: 55.3 KB


In [82]:
df_tweets.isnull().sum()

id                0
favorite_count    0
retweet_count     0
dtype: int64

In [83]:
df_tweets.shape

(2354, 3)

In [84]:
df_tweets.duplicated().sum()

0

## 2.4 Assessing conclusion

<h4>Archives</h4>
<h5>Quality issues</h5>
<ol>
<li>Timestamp column is a string. Needs to be changed to datetime</li>
<li>The source contains html residues.</li>
<li>We can observe that there are some 'None' objects present in"doggo", "floofer", "pupper" and "puppo" which have to be convert into 'NaN'.</li>
<li>The "name" column has a total of 745 'None' strings present.</li>
<li>The "name" column has a total of 87 stop words present</li>
<li>639 double links are present within "expanded_urls" column data</li>
<li>There are values not rated out of 10 as seen in the 'rating_denominator'. They are 23 in number</li>
</ol>

<h5>Tidiness issues</h5>
<ol>
<li>The variables doggo, floofer, pupper and puppo present in the data as depicted above, all represent one single variable and should be part of one single column</li>
<li>twitter_archives and df_tweets(the API data) form one observational unit</li>
</ol>


<h4>Image</h4>
<h5>Quality issues</h5>
<ol>
<li>We can observe that the columns "p1", "p2" and "p3" possess inconsistent writings</li>
</ol>


## 3. Cleaning Data